## Notebook Setup

In [1]:
import iex_cloud
import ishares_holdings
import json
import logging, log_setup
import utils
from typing import List, Optional, Tuple

In [2]:
logger = log_setup.init_notebook_log(logging.INFO)

<hr>

**Load Publishable Key from Local**

In [3]:
with open('../../config.json', 'r') as f:
    pk = json.load(f)['iex_pk']

<hr>

## Get ETF Constituents

In [4]:
ivv = 'https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf/'
ivv += '1467271812596.ajax?fileType=csv&fileName=IVV_holdings&dataType=fund'

In [5]:
fund_ref, fund_holdings = ishares_holdings.get(ivv)

**Verify Parsed Data**

In [6]:
fund_ref

{'name': 'iShares Core S&P 500 ETF',
 'holdings_date': datetime.date(2021, 12, 7),
 'inception': datetime.date(2000, 5, 15),
 'shares_outstanding': 701450000}

In [7]:
fund_holdings[:1]

,Ticker,Name,Sector,Asset Class,Market Value,Weight (%),Notional Value,Shares,Price,Location,Exchange,Currency,FX Rate,Market Currency,Accrual Date
0,AAPL,APPLE INC,Information Technology,Equity,2.231808e+10,6.74,2.231808e+10,130377822,171.18,United States,NASDAQ,USD,1.0,USD,None


In [8]:
fund_holdings[:3][['Ticker', 'Name', 'Sector', 'Weight (%)', 'Asset Class']]

,Ticker,Name,Sector,Weight (%),Asset Class
0,AAPL,APPLE INC,Information Technology,6.74,Equity
1,MSFT,MICROSOFT CORP,Information Technology,6.31,Equity
2,AMZN,AMAZON COM INC,Consumer Discretionary,3.85,Equity


In [9]:
fund_holdings[['Asset Class', 'Weight (%)']].groupby('Asset Class').sum().sort_values(by='Weight (%)')

,Weight (%)
Asset Class,
Futures,0.00
Cash Collateral and Margins,0.01
Cash,0.14
Money Market,0.14
Equity,99.86


<hr>

## Get OHLCV  (Open, High, Low, Close, Volume)

In [10]:
Date = str
Symbol = str
OHLCV = Tuple[Date, Symbol, float, float, float, float, float]

In [11]:

def get_ohlcv(api_token: str, ticker: str) -> Tuple[OHLCV, utils.Status]:
    """Get OHLCV for a single ticker."""
    r = iex_cloud.previous(api_token, ticker)
    ohlcv, status = parse_ohlcv(r.text)
    if status != utils.OK():
        logger.error(f'Error for {ticker}: {status.msg}')
    return ohlcv, status

In [12]:
def parse_ohlcv(s: str) -> Tuple[OHLCV, utils.Status]:
    """Parse response text into OHLC."""
    status: utils.Status = utils.OK()

    try:
        d = json.loads(s)
        ret = (d['date'],
               d['symbol'],
               d['open'],
               d['high'],
               d['low'],
               d['close'],
               d['volume'])
    except Exception as e:
        ret = ('', '', 0, 0, 0, 0, 0)
        status = utils.Error(f'{e}')

    return ret, status

**Success and Failure Examples**

In [13]:
get_ohlcv(pk, 'AAPL')

(('2021-12-07', 'AAPL', 169.08, 171.58, 168.34, 171.18, 120405352),
 OK(msg='OK'))

In [14]:
get_ohlcv(pk, 'UNICORN')

2021-12-08 13:41:54,728 - WARNING - utils - HTTP request returned code 404
2021-12-08 13:41:54,729 - ERROR - Notebook - Error for UNICORN: Expecting value: line 1 column 1 (char 0)


(('', '', 0, 0, 0, 0, 0),
 Error(msg='Expecting value: line 1 column 1 (char 0)'))

<hr>

## Joining the Data

In [15]:
df = fund_holdings[fund_holdings['Asset Class'] == 'Equity'] 
stocks = df[['Ticker', 'Name', 'Sector', 'Weight (%)']]
stocks[:3]

,Ticker,Name,Sector,Weight (%)
0,AAPL,APPLE INC,Information Technology,6.74
1,MSFT,MICROSOFT CORP,Information Technology,6.31
2,AMZN,AMAZON COM INC,Consumer Discretionary,3.85


In [16]:
Holding = Tuple[str, str, str, float]
PricedHolding = Tuple[Holding, OHLCV]

In [17]:
def add_ohlcv(api_token: str, holding: Holding) -> Optional[PricedHolding]:
    ohlcv, status = get_ohlcv(api_token, holding[0].strip())
    return (holding, ohlcv) if status == utils.OK() else None


In [18]:
for holding in stocks[:3].itertuples(index=False):
    print(add_ohlcv(pk, tuple(holding)))

(('AAPL', 'APPLE INC', 'Information Technology', 6.74), ('2021-12-07', 'AAPL', 169.08, 171.58, 168.34, 171.18, 120405352))
(('MSFT', 'MICROSOFT CORP', 'Information Technology', 6.31), ('2021-12-07', 'MSFT', 331.64, 335.8, 330.1, 334.92, 31021936))
(('AMZN', 'AMAZON COM INC', 'Consumer Discretionary', 3.85), ('2021-12-07', 'AMZN', 3492, 3549.99, 3466.69, 3523.29, 3320536))


<hr>

## Elm Code Gen

In [19]:
def module(name: str, exp: List[str]) -> str:
    """Module declaration."""
    e = exposing(exp)
    return f'module {name} {e}\n'

def exposing(names: List[str], exp_all: bool = False) -> str:
    """Exposing..."""
    if exp_all: return '(..)'
    xs = ', '.join(name for name in names)
    return f'exposing ( {xs} )'


In [20]:
module('MyModule', ['Foo', 'Bar'])

'module MyModule exposing ( Foo, Bar )\n'